In [1]:
import numpy as np
import os
import pandas as pd
import time
from itertools import product
import math

In [2]:
start = 1000000

end = 1250000


In [3]:
df_disease_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv")
df_drug_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv")
pos = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_pos.tsv")
neg = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_neg.tsv")

print len(pos)
print len(neg)

new_asso_li = list()
for index, row in neg[:1397].iterrows():
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    new_asso_li.append((query_drug, query_disease, query_asso))

new_pos_li = list()
for index2, row2 in pos.iterrows():
    pos_drug = row2['drug']
    pos_disease = row2['disease']
    new_pos_li.append((pos_drug, pos_disease)) 

print 'start : ' + time.strftime('%a %H:%M:%S')
desc_pair = list(product(new_asso_li, new_pos_li))
print 'end : ' + time.strftime('%a %H:%M:%S')

descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
for pair in desc_pair[start:end]:
    query_drug = pair[0][0]
    query_disease = pair[0][1]
    query_asso = pair[0][2]
    pos_drug = pair[1][0]
    pos_disease = pair[1][1]
    if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
    else:
        found_drug = df_drug_sim[(df_drug_sim.DrugBank_ID1 == query_drug) & (df_drug_sim.DrugBank_ID2 == pos_drug)]
        drug_cheSim = found_drug['CheSim'].values[0]
        drug_pheSim = found_drug['PheSim'].values[0]
    
        found_disease = df_disease_sim[(df_disease_sim.Disease_ID1 == query_disease) & (df_disease_sim.Disease_ID2 == pos_disease)]
        disease_pheSim = found_disease['PheSim'].values[0]
        disease_geSim = found_disease['GeSim'].values[0]
    
        drug_sim = [drug_cheSim, drug_pheSim]
        disease_sim = [disease_geSim, disease_pheSim]
        
        sim = [math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))]
        temp_df = pd.DataFrame(data = [[query_drug, query_disease, query_asso, sim[0], sim[1], sim[2], sim[3]]],
                           columns=['drug', 'disease', 'association', 'DrChe_DiGe', 'DrChe_DiPhe', 'DrPhe_DiGe', 'DrPhe_DiPhe'])
        descriptor = descriptor.append(temp_df)
        cnt = cnt +1
        if cnt % 10000 == 0:
            print "{} : ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')
print cnt
print descriptor.shape

real_desc = descriptor.groupby(['drug','disease']).max()
real_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_neg_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t', index=False)





1397
105763
start : Tue 11:40:26
end : Tue 11:40:26
start : Tue 11:40:26
10000 : Tue 11:47:37
20000 : Tue 11:54:49
30000 : Tue 12:02:03
40000 : Tue 12:09:40
50000 : Tue 12:17:14
60000 : Tue 12:24:46
70000 : Tue 12:32:11
80000 : Tue 12:39:39
90000 : Tue 12:47:18
100000 : Tue 12:54:52
110000 : Tue 13:02:57
120000 : Tue 13:11:15
130000 : Tue 13:19:18
140000 : Tue 13:27:23
150000 : Tue 13:35:22
160000 : Tue 13:43:38
170000 : Tue 13:52:08
180000 : Tue 14:00:25
190000 : Tue 14:09:08
200000 : Tue 14:18:04
210000 : Tue 14:26:49
220000 : Tue 14:35:42
230000 : Tue 14:44:07
240000 : Tue 14:52:46
250000 : Tue 15:01:50
end : Tue 15:01:50
250000
(250000, 7)


In [4]:
descriptor.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_neg_bef_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t', index=False)